In [1]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("../Datasets/XPR.csv")

In [3]:
df.head()

,Crypto,Date,Open,High,Low,Close
0,XRP,12/31/22,0.34,0.34,0.34,0.34
1,XRP,12/30/22,0.34,0.35,0.33,0.34
2,XRP,12/29/22,0.36,0.36,0.34,0.34
3,XRP,12/28/22,0.37,0.37,0.35,0.36
4,XRP,12/27/22,0.37,0.37,0.36,0.37


In [4]:
print("Dataframe Shape: " , df.shape)
print("Dataframe feature Datatypes: ", df.dtypes)
print("Dataframe Description: ", df.describe())

Dataframe Shape:  (1975, 6)
Dataframe feature Datatypes:  Crypto     object
Date       object
Open      float64
High      float64
Low       float64
Close     float64
dtype: object
Dataframe Description:                Open         High          Low        Close
count  1975.000000  1975.000000  1975.000000  1975.000000
mean      0.511838     0.533975     0.487266     0.510729
std       0.333890     0.358062     0.302996     0.330889
min       0.140000     0.150000     0.120000     0.140000
25%       0.290000     0.300000     0.280000     0.290000
50%       0.400000     0.410000     0.380000     0.400000
75%       0.630000     0.660000     0.600000     0.630000
max       2.780000     3.350000     2.550000     2.780000


In [6]:
df.isnull().sum()

Crypto    0
Date      0
Open      0
High      0
Low       0
Close     0
dtype: int64